In [1]:
import csv
import numpy as np
import textwrap

In [2]:
file_name = "./GSE/decompressed/GSE6186"

In [3]:
file = f"{file_name}.txt"
gem = []
sample_id = ["ID_REF"]
sample_title = [" "]
start = False
with open(file, 'r') as f:
    readCsv = csv.reader(f, delimiter='\t')
    for row in list(readCsv):
        if row:
            first = row[0]
            if first.startswith('!series_matrix_table_end'):
                break
            elif first.startswith('!series_matrix_table_begin'):
                start = True
            elif first.startswith('!Sample_title'):
                sample_title.extend(row[1:])
            elif first.startswith('!Sample_geo_accession'):
                sample_id.extend(row[1:])
            elif start:
                gem.append(row)

gem = np.array(gem, dtype='object')

In [4]:
# header = ['"'+f'{i}\n{j}'.strip()+'"' for i, j in zip(sample_id, sample_title)]
header = ['"{}"'.format(f'{i}\n{j}'.strip()) for i, j in zip(sample_id, sample_title)]

In [5]:
def get_max_len_column_value(data_array, column):
    # return len(max(data_array[:,[column]], key=len)[0])
    return len(max(data_array[:,column], key=len))

max_len = []
for i in range(gem.shape[1]):
    max_len.append(get_max_len_column_value(gem, i))

In [6]:
def fit(phrase, length):
    # https://stackoverflow.com/a/1166367
    w = textwrap.TextWrapper(width=length,break_long_words=False)
    # w = textwrap.TextWrapper(width=length,break_long_words=False,replace_whitespace=False)
    phrase = '\n'.join(w.wrap(phrase))
    return phrase

In [7]:
for i in range(gem.shape[1]):
    header[i] = fit(header[i], max_len[i]+4)
    # header[i] = fit(header[i], max_len[i])
header = np.array(header, dtype='object')

In [8]:
gem[0,:] = header # "ID_REF" "GSMXXX" "GSMXXX" -> "ID_REF" "GSMXXX desc ..." "GSMXXX desc ..."

In [9]:
# gem[0,:]
# header

# gem[1:,:]
# gem

In [10]:
file_name = file_name.replace('decompressed/','')
np.savetxt(f"{file_name}.tsv", gem, delimiter="\t", fmt='%s')